In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from suntimes import SunTimes  

In [8]:
identifying_columns = ['Referenzjahr', 'Stations_id']

In [4]:
ds_list = []
for variable_name in ['t2m', 't2min', 't2max', 'vpd', 'tp', 'ssrd']:
    ds_SSA = pd.read_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\Maize_ML_data_Africa_{variable_name}_pd_SOS3.csv').dropna(how='all')
    ds_SSA['observed time to beginning of flowering'] = pd.to_timedelta(ds_SSA['observed time to beginning of flowering']).dt.days#, 'D'
    ds_SSA = ds_SSA.loc[ds_SSA['observed time to beginning of flowering'] > 30] #30
    ds_list.append(ds_SSA)

In [9]:
ds_full = ds_list[0]
for variable_ind, variable_name in enumerate(['t2min', 't2max', 'vpd', 'tp', 'ssrd']):
    print(variable_name)
    ds_full = ds_full.merge(ds_list[variable_ind + 1][identifying_columns + [f'{variable_name} at day {n}' for n in range (300)]], on = identifying_columns)

t2min
t2max
vpd
tp
ssrd


In [12]:
def get_categories(anthesis_date):
    return np.array([anthesis_date < n for n in range(183)]).squeeze().astype(np.int64)
cats = ds_full[['observed time to beginning of flowering']].apply(get_categories, args = (), axis = 1)
cats_unlisted = cats.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
cats_unlisted = cats_unlisted.rename(columns = {n: f'dev stage at day {n}' for n in range(183)})
ds_full = ds_full.merge(cats_unlisted, left_index=True, right_index = True)

In [13]:
ds_full.loc[:, [f'DTF at day {n}' for n in range(193)]] = np.tile(ds_full['observed time to beginning of flowering'].values, (193, 1)).T - np.tile(np.arange(0, 193), (len(ds_full),1))

In [14]:
def get_photoperiods(inputs):
    #inputs = [lon, lat, day of start]
    day_of_start = datetime(inputs.iloc[2].year, inputs.iloc[2].month, inputs.iloc[2].day)
    lon = inputs.iloc[0]
    lat = inputs.iloc[1]
    sun = SunTimes(lon, lat) #can put altitude in also
    lengths = [(pd.to_datetime(sun.setutc(day_of_start + relativedelta(days=n))) - pd.to_datetime(sun.riseutc(day_of_start + relativedelta(days=n)))).seconds/3600 for n in range(170)]
    return np.array(lengths).squeeze().astype(np.float64)

In [18]:
def get_station_locations(dataset, ds_stations):
    ds_stations.index = ds_stations['LocationID']
    lat = [ds_stations._get_value(row, col) for row, col in zip(dataset['sitecode'], ['Latitude' for count in range(len(dataset))])] #station_data.lookup(row_labels = dataset['Stations_id'], col_labels = ['geograph.Breite'])
    lon = [ds_stations._get_value(row, col) for row, col in zip(dataset['sitecode'], ['Longitude' for count in range(len(dataset))])] #station_data._lookup(dataset['Stations_id'], ['geograph.Laenge'])
    dataset['lat'] = lat
    dataset['lon'] = lon
    dataset['lat'] = dataset['lat'].map(lambda x: x[0] if isinstance(x, float) == False else x)
    dataset['lon'] = dataset['lon'].map(lambda x: x[0] if isinstance(x, float) == False else x)
    return dataset

In [20]:
stations_data_lobell = pd.read_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\African_data\\Lobell2011\\EIL_site_latlon.csv')
ds_full['sitecode'] = ds_full['Stations_id']
ds_full = get_station_locations(ds_full, stations_data_lobell)

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_16712\3268591058.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ds_full['sitecode'] = ds_full['Stations_id']
C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_16712\4157498997.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset['lat'] = lat
C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_16712\4157498997.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

In [22]:
ds_full['Planting date'] = pd.to_datetime(ds_full['WC SOS date'])
photoperiods = ds_full[['lon', 'lat', 'Planting date']].apply(get_photoperiods, args = (), axis = 1)
photoperiods_unlisted = photoperiods.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
photoperiods_unlisted = photoperiods_unlisted.rename(columns = {n: f'photoperiod at day {n}' for n in range(170)})
ds_full = ds_full.merge(photoperiods_unlisted, left_index=True, right_index = True)

In [23]:
ds_full.to_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\saved_ML_datasets\\SSA_ML_ERA5_1999_2008.csv')

In [2]:
ds_t2m_DE = pd.read_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\Maize_ML_data_ERA5_t2m_pd_SOS.csv').dropna()
ds_t2m_DE['observed time to beginning of flowering'] = pd.to_timedelta(ds_t2m_DE['observed time to beginning of flowering']).dt.days#, 'D'
ds_t2m_DE = ds_t2m_DE.loc[ds_t2m_DE['observed time to beginning of flowering'] > 30] #30

In [14]:
ds_list = []
for variable_name in ['t2m', 't2min', 't2max', 'vpd', 'tp', 'ssrd']:
    ds_DE = pd.read_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\Maize_ML_data_ERA5_{variable_name}_pd_SOS.csv').dropna(how='all')
    ds_DE['observed time to beginning of flowering'] = pd.to_timedelta(ds_DE['observed time to beginning of flowering']).dt.days#, 'D'
    ds_DE = ds_DE.loc[ds_DE['observed time to beginning of flowering'] > 30] #30
    ds_list.append(ds_DE)

In [15]:
ds_full = ds_list[0]
for variable_ind, variable_name in enumerate(['t2min', 't2max', 'vpd', 'tp', 'ssrd']):
    ds_full = ds_full.merge(ds_list[variable_ind + 1][identifying_columns + [f'{variable_name} at day {n}' for n in range (310)]], on = identifying_columns)

In [17]:
def get_categories(anthesis_date):
    return np.array([anthesis_date < n for n in range(183)]).squeeze().astype(np.int64)
cats = ds_full[['observed time to beginning of flowering']].apply(get_categories, args = (), axis = 1)
cats_unlisted = cats.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
cats_unlisted = cats_unlisted.rename(columns = {n: f'dev stage at day {n}' for n in range(183)})
ds_full = ds_full.merge(cats_unlisted, left_index=True, right_index = True)

In [19]:
ds_full.loc[:, [f'DTF at day {n}' for n in range(193)]] = np.tile(ds_full['observed time to beginning of flowering'].values, (193, 1)).T - np.tile(np.arange(0, 193), (len(ds_full),1))

In [20]:
def get_photoperiods(inputs):
    #inputs = [lon, lat, day of start]
    day_of_start = datetime(inputs.iloc[2].year, inputs.iloc[2].month, inputs.iloc[2].day)
    lon = inputs.iloc[0]
    lat = inputs.iloc[1]
    sun = SunTimes(lon, lat) #can put altitude in also
    lengths = [(pd.to_datetime(sun.setutc(day_of_start + relativedelta(days=n))) - pd.to_datetime(sun.riseutc(day_of_start + relativedelta(days=n)))).seconds/3600 for n in range(170)]
    return np.array(lengths).squeeze().astype(np.float64)

In [23]:
ds_full['Planting date'] = pd.to_datetime(ds_full['Planting date'])
photoperiods = ds_full[['lon', 'lat', 'Planting date']].apply(get_photoperiods, args = (), axis = 1)
photoperiods_unlisted = photoperiods.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
photoperiods_unlisted = photoperiods_unlisted.rename(columns = {n: f'photoperiod at day {n}' for n in range(170)})
ds_full = ds_full.merge(photoperiods_unlisted, left_index=True, right_index = True)

In [24]:
ds_full

,Unnamed: 0,Planting date,Referenzjahr,Stations_id,lat,lon,Objekt_id,Reporting method,Historic or recent,ERNTEVERFAHREN_ID,...,photoperiod at day 160,photoperiod at day 161,photoperiod at day 162,photoperiod at day 163,photoperiod at day 164,photoperiod at day 165,photoperiod at day 166,photoperiod at day 167,photoperiod at day 168,photoperiod at day 169
0,0,2004-05-03,2004,7504,54.3167,10.0500,215,annual,historic,5.0,...,10.916667,10.850000,10.766667,10.700000,10.633333,10.550000,10.483333,10.400000,10.333333,10.266667
1,1,2005-04-25,2005,7504,54.3167,10.0500,215,annual,historic,5.0,...,11.500000,11.433333,11.350000,11.283333,11.233333,11.150000,11.083333,11.000000,10.933333,10.850000
2,2,2007-04-20,2007,7504,54.3167,10.0500,215,annual,historic,5.0,...,11.900000,11.833333,11.750000,11.700000,11.616667,11.550000,11.466667,11.400000,11.316667,11.266667
3,3,2008-04-28,2008,7504,54.3167,10.0500,215,annual,historic,5.0,...,11.266667,11.200000,11.133333,11.050000,11.000000,10.916667,10.850000,10.766667,10.700000,10.633333
4,4,2009-05-04,2009,7504,54.3167,10.0500,215,annual,historic,5.0,...,10.850000,10.783333,10.716667,10.650000,10.583333,10.516667,10.433333,10.366667,10.300000,10.216667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15303,84738,2021-04-30,2021,19914,50.3168,9.3627,215,annual,historic,5.0,...,11.283333,11.216667,11.166667,11.083333,11.033333,10.966667,10.900000,10.850000,10.783333,10.733333
15304,84739,2022-05-06,2022,19914,50.3168,9.3627,215,annual,historic,5.0,...,10.933333,10.866667,10.800000,10.733333,10.666667,10.616667,10.550000,10.483333,10.433333,10.366667
15305,84740,2015-04-24,2015,19915,48.3756,9.4963,210,immediate,historic,NaN,...,11.700000,11.650000,11.583333,11.533333,11.483333,11.416667,11.366667,11.300000,11.250000,11.183333
15306,84741,2016-05-06,2016,19915,48.3756,9.4963,210,immediate,historic,NaN,...,10.966667,10.900000,10.850000,10.783333,10.750000,10.683333,10.633333,10.566667,10.516667,10.450000


In [25]:
ds_full.to_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\saved_ML_datasets\\DE_ML_ERA5_2001_2024.csv')